In [1]:
# load all required libraries for Dog's Breed Identification Project
import cv2
import numpy as np 
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

In [2]:
#read the csv file
df_labels = pd.read_csv("D:\\Dog Breed Classification\\Data Set\\labels.csv")
#store training and testing images folder location
train_file = "D:\\Dog Breed Classification\\Data Set\\train\\"
test_file = "D:\\Dog Breed Classification\\Data Set\\test\\"

In [3]:
print(df_labels)

                                     id                     breed
0      000bec180eb18c7604dcecc8fe0dba07               boston_bull
1      001513dfcb2ffafc82cccf4d8bbaba97                     dingo
2      001cdf01b096e06d78e9e5112d419397                  pekinese
3      00214f311d5d2247d5dfe4fe24b2303d                  bluetick
4      0021f9ceb3235effd7fcde7f7538ed62          golden_retriever
...                                 ...                       ...
10217  ffd25009d635cfd16e793503ac5edef0                    borzoi
10218  ffd3f636f7f379c51ba3648a9ff8254f            dandie_dinmont
10219  ffe2ca6c940cddfee68fa3cc6c63213f                  airedale
10220  ffe5f6d8e2bff356e9482a80a6e29aac        miniature_pinscher
10221  fff43b07992508bc822f33d8ffd902ae  chesapeake_bay_retriever

[10222 rows x 2 columns]


In [4]:
    print("Total number of unique Dog Breeds :",len(df_labels.breed.unique()))

Total number of unique Dog Breeds : 120


In [5]:
#specify number
num_breeds = 120
im_size = 224
batch_size = 64
encoder = LabelEncoder()

In [6]:
#get all 120 unique breeds record 
breed_dict = list(df_labels['breed'].value_counts().keys()) 
new_list = sorted(breed_dict,reverse=True)[:num_breeds+1:1]
#change the dataset to have those 120 unique breed records
df_labels = df_labels.query('breed in @new_list')

In [7]:
#create new column which will contain image name with the image extension
df_labels['img_file'] = df_labels['id'].apply(lambda x: x + ".jpg")
print(df_labels['img_file'])

0        000bec180eb18c7604dcecc8fe0dba07.jpg
1        001513dfcb2ffafc82cccf4d8bbaba97.jpg
2        001cdf01b096e06d78e9e5112d419397.jpg
3        00214f311d5d2247d5dfe4fe24b2303d.jpg
4        0021f9ceb3235effd7fcde7f7538ed62.jpg
                         ...                 
10217    ffd25009d635cfd16e793503ac5edef0.jpg
10218    ffd3f636f7f379c51ba3648a9ff8254f.jpg
10219    ffe2ca6c940cddfee68fa3cc6c63213f.jpg
10220    ffe5f6d8e2bff356e9482a80a6e29aac.jpg
10221    fff43b07992508bc822f33d8ffd902ae.jpg
Name: img_file, Length: 10222, dtype: object


In [27]:
#create a numpy array of the shape
#(number of dataset records, image size , image size, 3 for rgb channel ayer)
#this will be input for model
train_x = np.zeros((len(df_labels), im_size, im_size, 3), dtype='float32')
 
#iterate over img_file column of our dataset
for i, img_id in enumerate(df_labels['img_file']):
  #read the image file and convert into numeric format
  #resize all images to one dimension i.e. 224x224
  #we will get array with the shape of
  # (224,224,3) where 3 is the RGB channels layers
  img = cv2.resize(cv2.imread(train_file+img_id,cv2.IMREAD_COLOR),((im_size,im_size)))
  #scale array into the range of -1 to 1.
  #preprocess the array and expand its dimension on the axis 0 
  img_array = preprocess_input(np.expand_dims(np.array(img[...,::-1].astype(np.float32)).copy(), axis=0))
  #update the train_x variable with new element
  train_x[i] = img_array
  

In [9]:
#This will be the target for the model.
#convert breed names into numerical format
train_y = encoder.fit_transform(df_labels["breed"].values)

In [10]:
#split the dataset in the ratio of 80:20. 
#80% for training and 20% for testing purpose
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y,test_size=0.2,random_state=42)

In [11]:
#Image augmentation using ImageDataGenerator class
train_datagen = ImageDataGenerator(rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
 
#generate images for training sets 
train_generator = train_datagen.flow(x_train, 
                                     y_train, 
                                     batch_size=batch_size)
 
#same process for Testing sets also by declaring the instance
test_datagen = ImageDataGenerator()
 
test_generator = test_datagen.flow(x_test, 
                                     y_test, 
                                     batch_size=batch_size)

In [30]:
#building the model using ResNet50V2 with input shape of our image array
#weights for our network will be from of imagenet dataset
#we will not include the first Dense layer
resnet = ResNet50V2(input_shape = [im_size,im_size,3], weights='imagenet', include_top=False)
#freeze all trainable layers and train only top layers 
for layer in resnet.layers:
    layer.trainable = False
 
#add global average pooling layer and Batch Normalization layer
x = resnet.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
#add fully connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

In [13]:
#add output layer having the shape equal to number of breeds
predictions = Dense(num_breeds, activation='softmax')(x)
 
#create model class with inputs and outputs
model = Model(inputs=resnet.input, outputs=predictions)
#model.summary()

In [14]:
#epochs for model training and learning rate for optimizer
epochs = 20
learning_rate = 1e-3
 
#using RMSprop optimizer to compile or build the model
optimizer = RMSprop(learning_rate=learning_rate,rho=0.9)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])
 
#fit the training generator data and train the model
hist = model.fit(train_generator,
                 steps_per_epoch= x_train.shape[0] // batch_size,
                 epochs= epochs,
                 validation_data= test_generator,
                 validation_steps= x_test.shape[0] // batch_size)
 
#Save the model for prediction
model.save("model")

Epoch 1/20
127/127 [==============================] - 767s 6s/step - loss: 2.7246 - accuracy: 0.3533 - val_loss: 1.0618 - val_accuracy: 0.6799
Epoch 2/20
127/127 [==============================] - 1334s 11s/step - loss: 1.6569 - accuracy: 0.5454 - val_loss: 0.9375 - val_accuracy: 0.7162
Epoch 3/20
127/127 [==============================] - 813s 6s/step - loss: 1.4814 - accuracy: 0.5881 - val_loss: 0.8802 - val_accuracy: 0.7440
Epoch 4/20
127/127 [==============================] - 820s 6s/step - loss: 1.4226 - accuracy: 0.6029 - val_loss: 0.8897 - val_accuracy: 0.7334
Epoch 5/20
127/127 [==============================] - 786s 6s/step - loss: 1.3657 - accuracy: 0.6227 - val_loss: 0.8896 - val_accuracy: 0.7424
Epoch 6/20
127/127 [==============================] - 809s 6s/step - loss: 1.2911 - accuracy: 0.6345 - val_loss: 0.9150 - val_accuracy: 0.7369
Epoch 7/20
127/127 [==============================] - 822s 6s/step - loss: 1.2834 - accuracy: 0.6391 - val_loss: 0.9021 - val_accuracy: 0.74

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [31]:
#load the model
model = load_model("model")
 
#get the image of the dog for prediction
pred_img_path = "C:\\Users\\asus\\Dropbox\\My PC (LAPTOP-KVI3FLV1)\\Downloads\\Pooh.jpg"
#read the image file and convert into numeric format
#resize all images to one dimension i.e. 224x224
pred_img_array = cv2.resize(cv2.imread(pred_img_path,cv2.IMREAD_COLOR),((im_size,im_size)))
#scale array into the range of -1 to 1.
#expand the dimension on the axis 0 and normalize the array values
pred_img_array = preprocess_input(np.expand_dims(np.array(pred_img_array[...,::-1].astype(np.float32)).copy(), axis=0))
 
#feed the model with the image array for prediction
pred_val = model.predict(np.array(pred_img_array,dtype="float32"))
 
#display the image of dog
cv2.imshow("Dog Image",cv2.resize(cv2.imread(pred_img_path,cv2.IMREAD_COLOR),((im_size,im_size)))) 
 
#display the predicted breed of dog
pred_breed = sorted(new_list)[np.argmax(pred_val)]
print("Predicted Breed for this Dog is :",pred_breed)

1/1 [==============================] - 3s 3s/step
Predicted Breed for this Dog is : afghan_hound
